In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Autoinflammatory_Disorders"
cohort = "GSE43553"

# Input paths
in_trait_dir = "../../input/GEO/Autoinflammatory_Disorders"
in_cohort_dir = "../../input/GEO/Autoinflammatory_Disorders/GSE43553"

# Output paths
out_data_file = "../../output/preprocess/Autoinflammatory_Disorders/GSE43553.csv"
out_gene_data_file = "../../output/preprocess/Autoinflammatory_Disorders/gene_data/GSE43553.csv"
out_clinical_data_file = "../../output/preprocess/Autoinflammatory_Disorders/clinical_data/GSE43553.csv"
json_path = "../../output/preprocess/Autoinflammatory_Disorders/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Microarray-based gene expression profiling in patients with cryopyrin-associated periodic syndromes defines a disease-related signature and IL-1-responsive transcripts."
!Series_summary	"To analyse gene expression patterns and to define a specific gene expression signature in patients with the severe end of the spectrum of cryopyrin-associated periodic syndromes (CAPS). The molecular consequences of interleukin 1 inhibition were examined by comparing gene expression patterns in 16 CAPS patients before and after treatment with anakinra."
!Series_summary	"Many DEG include transcripts related to the regulation of innate and adaptive immune responses, oxidative stress, cell death, cell adhesion and motility. A set of gene expression-based models comprising the CAPS-specific gene expression signature correctly classified all 17 samples from an independent dataset. This classifier also correctly identified 15 of 16 post-anakinra CAPS samples despite the

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Dict, Any, Optional, Callable
import json
import re

# 1. Analyzing the availability of gene expression data
# Based on the background information, this dataset contains gene expression data from microarray analysis
is_gene_available = True

# 2. Analyzing clinical data availability and defining conversion functions

# 2.1 For trait (Autoinflammatory Disorders):
# Row 3 contains 'disease state: CAPS', 'disease state: other autoinflammatory disease', etc.
trait_row = 3

# Function to convert trait values
def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: CAPS (1) vs other conditions (0)
    if 'CAPS' in value:
        return 1
    elif 'other autoinflammatory disease' in value or 'healthy' in value:
        return 0
    else:
        return None

# 2.2 For age:
# No clear data for age in the sample characteristics
age_row = None

def convert_age(value):
    # This function is defined but won't be used since age data is not available
    if value is None or pd.isna(value):
        return None
    
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# 2.3 For gender:
# No clear data for gender in the sample characteristics
gender_row = None

def convert_gender(value):
    # This function is defined but won't be used since gender data is not available
    if value is None or pd.isna(value):
        return None
    
    if ':' in str(value):
        value = value.split(':', 1)[1].strip().lower()
    
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    else:
        return None

# 3. Save metadata for initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait data is available)
if trait_row is not None:
    try:
        # The sample characteristics dictionary from the previous output just 
        # provides a summary of unique values, and isn't the actual clinical data.
        # Since we don't have the actual clinical data file, we'll skip this step.
        print("Clinical data extraction requires the actual clinical data file, which is not available.")
        print("Trait information is available (trait_row = 3), but we need the complete clinical dataset.")
        print("The rest of the preprocessing pipeline will continue without the clinical data extraction step.")
    except Exception as e:
        print(f"Error processing clinical data: {e}")


Clinical data extraction requires the actual clinical data file, which is not available.
Trait information is available (trait_row = 3), but we need the complete clinical dataset.
The rest of the preprocessing pipeline will continue without the clinical data extraction step.


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These gene identifiers appear to be Affymetrix probe IDs (e.g., "1007_s_at", "1053_at"), not standard human gene symbols.
# Affymetrix IDs need to be mapped to human gene symbols for biological interpretation.
# The format with "_at" suffix is characteristic of Affymetrix microarray probe IDs.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation containing probe IDs and gene symbols
# 'ID' column stores the same kind of gene identifiers (Affymetrix IDs) as in gene_data
# 'Gene Symbol' column stores the corresponding gene symbols
probe_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get the gene mapping dataframe by extracting these two columns
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_col)
print(f"Gene mapping dataframe created with {len(mapping_df)} probe-to-gene mappings")
print(f"Sample of mapping data (first 5 rows):")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data created with {len(gene_data)} genes and {gene_data.shape[1]} samples")
print(f"First few gene symbols in the processed gene data:")
print(gene_data.index[:10])


Gene mapping dataframe created with 21225 probe-to-gene mappings
Sample of mapping data (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Gene expression data created with 13830 genes and 100 samples
First few gene symbols in the processed gene data:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB',
       'AAK1', 'AAMDC'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# First, normalize gene symbols using the function from the library
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene count: {len(gene_data)}")
print(f"Normalized gene count: {len(normalized_gene_data)}")

# Create directory for the gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data to a CSV file
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. We need to first load or recreate the selected_clinical_df
try:
    # Try to load the previously saved clinical data
    selected_clinical_df = pd.read_csv(out_clinical_data_file)
except:
    # If loading fails, recreate the clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)

    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)

# Check if linking was successful
if len(linked_data) == 0 or trait not in linked_data.columns:
    print("Linking clinical and genetic data failed - no valid rows or trait column missing")
    # Set is_usable to False and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Consider it biased if linking fails
        df=pd.DataFrame({trait: [], 'Gender': []}), 
        note="Data linking failed - unable to process gene expression data appropriately."
    )
    print("The dataset was determined to be not usable for analysis.")
else:
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
    if trait_type == 'binary':
        if len(linked_data[trait].value_counts()) >= 2:
            is_trait_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            print(f"Trait '{trait}' has only one unique value, considering it biased.")
            is_trait_biased = True
    else:
        is_trait_biased = judge_continuous_variable_biased(linked_data, trait)
    
    # Remove biased demographic features
    unbiased_linked_data = linked_data.copy()
    if 'Age' in unbiased_linked_data.columns:
        age_biased = judge_continuous_variable_biased(unbiased_linked_data, 'Age')
        if age_biased:
            print(f"The distribution of the feature \'Age\' in this dataset is severely biased.")
            unbiased_linked_data = unbiased_linked_data.drop(columns=['Age'])
    
    if 'Gender' in unbiased_linked_data.columns:
        if len(unbiased_linked_data['Gender'].value_counts()) >= 2:
            gender_biased = judge_binary_variable_biased(unbiased_linked_data, 'Gender')
            if gender_biased:
                print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.")
                unbiased_linked_data = unbiased_linked_data.drop(columns=['Gender'])
        else:
            print(f"Gender has only one unique value, considering it biased and removing.")
            unbiased_linked_data = unbiased_linked_data.drop(columns=['Gender'])
    
    # 5. Conduct quality check and save the cohort information.
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=unbiased_linked_data, 
        note="Dataset contains gene expression data from whole blood of systemic juvenile idiopathic arthritis (SJIA) patients treated with canakinumab or placebo and healthy controls."
    )
    
    # 6. If the linked data is usable, save it as a CSV file.
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("The dataset was determined to be not usable for analysis due to bias in the trait distribution.")

Original gene count: 13830
Normalized gene count: 13542


Normalized gene data saved to ../../output/preprocess/Autoinflammatory_Disorders/gene_data/GSE43553.csv
Preview of selected clinical features:
{'GSM1065191': [1.0], 'GSM1065192': [1.0], 'GSM1065193': [1.0], 'GSM1065194': [1.0], 'GSM1065195': [1.0], 'GSM1065196': [1.0], 'GSM1065197': [1.0], 'GSM1065198': [1.0], 'GSM1065199': [1.0], 'GSM1065200': [1.0], 'GSM1065201': [1.0], 'GSM1065202': [1.0], 'GSM1065203': [1.0], 'GSM1065204': [1.0], 'GSM1065205': [1.0], 'GSM1065206': [1.0], 'GSM1065207': [1.0], 'GSM1065208': [1.0], 'GSM1065209': [1.0], 'GSM1065210': [1.0], 'GSM1065211': [1.0], 'GSM1065212': [1.0], 'GSM1065213': [1.0], 'GSM1065214': [nan], 'GSM1065215': [nan], 'GSM1065216': [nan], 'GSM1065217': [nan], 'GSM1065218': [nan], 'GSM1065219': [nan], 'GSM1065220': [nan], 'GSM1065221': [nan], 'GSM1065222': [nan], 'GSM1065223': [nan], 'GSM1065224': [nan], 'GSM1065225': [nan], 'GSM1065226': [nan], 'GSM1065227': [nan], 'GSM1065228': [nan], 'GSM1065229': [nan], 'GSM1065230': [nan], 'GSM1065231': [n

Linked data shape after handling missing values: (66, 13543)
For the feature 'Autoinflammatory_Disorders', the least common label is '1.0' with 23 occurrences. This represents 34.85% of the dataset.
A new JSON file was created at: ../../output/preprocess/Autoinflammatory_Disorders/cohort_info.json


Linked data saved to ../../output/preprocess/Autoinflammatory_Disorders/GSE43553.csv
